In [ ]:
#pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')


Saving TheAppropriationBill_2023.pdf to TheAppropriationBill_2023 (1).pdf
User uploaded file "TheAppropriationBill_2023 (1).pdf" with length 915647 bytes


In [ ]:
# laoding the document
loader = UnstructuredFileLoader("/content/TheAppropriationBill_2023.pdf")
documents = loader.load()

In [ ]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

list

In [ ]:
len(texts)

34

In [ ]:
#texts[:3]

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
persist_directory = "doc_db2"

In [ ]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
import shutil

folder_name = "doc_db2"  # Replace with your folder name
zip_file_name = "approprationbill.zip"  # Name of the zip file to create

shutil.make_archive(folder_name, 'zip', folder_name)


'/content/doc_db2.zip'

In [ ]:
from google.colab import files

files.download('/content/doc_db2.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# invoke the qa chain and get a response for user query
query = "What is the purpose of this paper?"
response = qa_chain.invoke({"query": query})

In [ ]:
print(response)

{'query': 'What is the purpose of this paper?', 'result': 'The purpose of this paper is to provide for the appropriation of monies for the 2023/2024 financial year in Kenya. Specifically, it is the Appropriation Bill, 2023, which aims to authorize the issuance of a sum of money out of the Consolidated Fund and its application towards the service of the year ending on the 30th June, 2024, and to appropriate that sum and a sum voted on account by the National Assembly for certain public services and purposes.', 'source_documents': [Document(metadata={'source': '/content/TheAppropriationBill_2023.pdf'}, page_content='3. In addition to the sum granted by section 2, there may be applied, for the several services and purposes specified in the Schedule, the sums specified out of any money directed to be applied as appropriations in aid under Article 206(1)(b) of the Constitution.\n\n4. The disbursement of the monies appropriated to the State Department for Higher Education and Research under 

In [ ]:
print(response["result"])

The purpose of this paper is to provide for the appropriation of monies for the 2023/2024 financial year in Kenya. Specifically, it is the Appropriation Bill, 2023, which aims to authorize the issuance of a sum of money out of the Consolidated Fund and its application towards the service of the year ending on the 30th June, 2024, and to appropriate that sum and a sum voted on account by the National Assembly for certain public services and purposes.


In [ ]:
print(response["source_documents"][0].metadata["source"])

/content/TheAppropriationBill_2023.pdf


In [ ]:
# invoke the qa chain and get a response for user query
query = "how has money been allocated according to this document?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

According to the document, money has been allocated to various government departments and programs for the financial year ending June 30, 2024. The allocations are categorized into recurrent expenditure (current expenses) and development expenditure (capital expenses).

Here are some examples of allocations:

**Recurrent Expenditure:**

* Executive Office of the President: KSh 2,466,400,000
* Office of the Deputy President: KSh 569,649,224
* State Department for Social Protection and Senior Citizens Affairs: KSh 32,442,180,000
* State Department for Mining: KSh 1,365,200,000
* State Department for Petroleum: KSh 342,960,000
* State Department for Tourism: KSh 1,087,900,000

**Development Expenditure:**

* Office of the Teachers Service Commission: KSh 1,139,000,000
* Office of the Auditor General: KSh 310,000,000
* National Gender and Equality Commission: KSh 4,044,800
* State Department for Industry: KSh 340,000,000
* State Department for Micro, Small and Medium Enterprises Developmen

In [ ]:
# invoke the qa chain and get a response for user query
query = "how many much money has been allocated to health?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

Based on the provided context, I can see several allocations for health-related programs. Here are a few:

* For the State Department for Medical Services:
	+ Capital expenses: 12,704,000,000 (D1082)
	+ Current expenses: 43,517,520,000 (R1082)
* For the State Department for Public Health and Professional Standards:
	+ Capital expenses: 1,200,000,000 (D1083)
	+ Current expenses: 11,924,600,000 (R1083)

To give you a total, I'll add up these allocations:

* Capital expenses: 12,704,000,000 + 1,200,000,000 = 13,904,000,000
* Current expenses: 43,517,520,000 + 11,924,600,000 = 55,442,120,000

So, the total allocation for health is approximately 69,346,120,000.
******************************
Source Document: /content/TheAppropriationBill_2023.pdf
